In [3]:
import psycopg2
import pandas as pd
import plotly.tools as tls
import plotly.express as px
from statsmodels.tsa.seasonal import seasonal_decompose
from dotenv import load_dotenv
from os import getenv

In [ ]:
load_dotenv()
connection = psycopg2.connect(database=getenv("db"),
                            user=getenv("user"),
                            password=getenv("pwd"),
                            host="localhost", port=5432)

query = """
select
	at.fecha,
	sum(at.afluencia_tren_ligero) as afluencia_tren_ligero,
	sum(at.afluencia_trolebus) as afluencia_trolebus,
	sum(at.afluencia_cablebus) as afluencia_cablebus,
	LTRIM(RTRIM(TO_CHAR(at.fecha, 'Day'))) as dia_nombre,
	DATE_PART('day', at.fecha) as dia,
	DATE_PART('month', at.fecha) as mes,
	case when cdf.dia is not null then true else false end as es_festivo
from vw_afluencia_transportes at
left join cat_dias_festivos cdf
	on cdf.dia = DATE_PART('day', at.fecha) and cdf.mes = DATE_PART('month', at.fecha)
group by 
	at.fecha, cdf.dia, cdf.mes
order by 
	at.fecha asc
"""

df = pd.read_sql(query, con=connection)

df['fecha'] = pd.to_datetime(df["fecha"])

df.head()

C:\Users\Ian\AppData\Local\Temp\ipykernel_2276\1681080381.py:26: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,fecha,afluencia_tren_ligero,afluencia_trolebus,afluencia_cablebus,dia_nombre,dia,mes,es_festivo
0,2022-01-01,16057.0,28086.0,73120.0,Saturday,1.0,1.0,True
1,2022-01-02,24134.0,48312.0,87359.0,Sunday,2.0,1.0,False
2,2022-01-03,55064.0,75247.0,104001.0,Monday,3.0,1.0,False
3,2022-01-04,58005.0,78454.0,108199.0,Tuesday,4.0,1.0,False
4,2022-01-05,57677.0,80054.0,117619.0,Wednesday,5.0,1.0,False


In [3]:
px.scatter(df[['fecha','afluencia_trolebus', 'es_festivo']],
        x='fecha', y='afluencia_trolebus',
        color='es_festivo', color_discrete_sequence=['Red', 'Blue'])

In [4]:
decomposition = seasonal_decompose(
    df[['fecha','afluencia_trolebus']].set_index(['fecha']),
    model='additive', period=7)
tls.mpl_to_plotly(decomposition.plot())

In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

codificador_dia_nombre = LabelEncoder()
codificador_dia_nombre.fit_transform(df.dia_nombre)

array([2, 3, 1, ..., 0, 2, 3], shape=(1339,))

In [6]:
df_train = df[['fecha', 'afluencia_tren_ligero', 'dia_nombre', 'es_festivo']].copy()
df_train['dia_nombre'] = codificador_dia_nombre.fit_transform(df.dia_nombre)
df_train['es_festivo'] = df_train['es_festivo'].astype(int)
df_train['year'] = df_train['fecha'].dt.year
df_train['afluencia_tren_ligero'] = df_train['afluencia_tren_ligero'].astype(int)
df_train['sin_dayofyear'] = np.sin(2 * np.pi * df['fecha'].dt.dayofyear / 365)
df_train['cos_dayofyear'] = np.cos(2 * np.pi * df['fecha'].dt.dayofyear / 365)

df_train

,fecha,afluencia_tren_ligero,dia_nombre,es_festivo,year,sin_dayofyear,cos_dayofyear
0,2022-01-01,16057,2,1,2022,0.017213,0.999852
1,2022-01-02,24134,3,0,2022,0.034422,0.999407
2,2022-01-03,55064,1,0,2022,0.051620,0.998667
3,2022-01-04,58005,5,0,2022,0.068802,0.997630
4,2022-01-05,57677,6,0,2022,0.085965,0.996298
...,...,...,...,...,...,...,...
1334,2025-08-27,94699,6,0,2025,-0.826354,-0.563151
1335,2025-08-28,96764,4,0,2025,-0.835925,-0.548843
1336,2025-08-29,96028,0,0,2025,-0.845249,-0.534373
1337,2025-08-30,61700,2,0,2025,-0.854322,-0.519744


In [7]:
ventana = 5

def crear_ventanas(df, target_col='afluencia_tren_ligero', n_lags=5):
    for lag in range(1, n_lags + 1):
        df[f'{target_col}_lag_{lag}'] = df[target_col].shift(lag)
    df = df.dropna()
    return df

df_train_lags = crear_ventanas(df_train.copy(), 'afluencia_tren_ligero', n_lags=ventana)
df_train_lags.head()

,fecha,afluencia_tren_ligero,dia_nombre,es_festivo,year,sin_dayofyear,cos_dayofyear,afluencia_tren_ligero_lag_1,afluencia_tren_ligero_lag_2,afluencia_tren_ligero_lag_3,afluencia_tren_ligero_lag_4,afluencia_tren_ligero_lag_5
5,2022-01-06,50310,4,0,2022,0.103102,0.994671,57677.0,58005.0,55064.0,24134.0,16057.0
6,2022-01-07,55300,0,0,2022,0.120208,0.992749,50310.0,57677.0,58005.0,55064.0,24134.0
7,2022-01-08,43929,2,0,2022,0.137279,0.990532,55300.0,50310.0,57677.0,58005.0,55064.0
8,2022-01-09,22377,3,0,2022,0.154309,0.988023,43929.0,55300.0,50310.0,57677.0,58005.0
9,2022-01-10,56514,1,0,2022,0.171293,0.985220,22377.0,43929.0,55300.0,50310.0,57677.0


In [8]:
X = df_train_lags.drop(columns=['fecha', 'afluencia_tren_ligero'])
y = df_train_lags['afluencia_tren_ligero']

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [9]:
# Creamos un DataFrame con ambas series y las fechas
df_plot = pd.DataFrame({
    'fecha': df_train_lags.loc[X_test.index, 'fecha'].dt.date.values,
    'Real': y_test.values,
    'Predicho': y_pred
})

# Transformamos a formato "largo" para graficar con Plotly Express
df_melt = df_plot.melt(id_vars='fecha', value_vars=['Real', 'Predicho'], var_name='Tipo', value_name='Valor')

# Graficamos
fig = px.line(df_melt, x='fecha', y='Valor', color='Tipo',
				title='Valores reales vs predichos',
				labels={'fecha': 'Fecha', 'Valor': 'Valor'})
fig.show()

In [10]:
# 📊 Métricas de evaluación
mae = mean_absolute_error(y_test.values, y_pred)
rmse = np.sqrt(mean_squared_error(y_test.values, y_pred))
r2 = r2_score(y_test.values, y_pred)

print(f"MAE (Error Absoluto Medio): {mae:.4f}")
print(f"RMSE (Raíz del Error Cuadrático Medio): {rmse:.4f}")
print(f"R² (Coeficiente de Determinación): {r2:.4f}")

MAE (Error Absoluto Medio): 9661.0205
RMSE (Raíz del Error Cuadrático Medio): 13919.8296
R² (Coeficiente de Determinación): 0.7884


In [11]:
forecasting = df_train.tail(ventana)
longitud = 120
for i in range(longitud):
    fecha = forecasting.loc[forecasting.index[-1], 'fecha']+pd.Timedelta(days=1)
    forecasting.loc[forecasting.index[-1]+1, :] = [
		fecha,
		0,
		codificador_dia_nombre.transform([fecha.day_name()])[0],
		0,
		fecha.year,
		np.sin(2 * np.pi * fecha.day_of_year / 365),
		np.cos(2 * np.pi * fecha.day_of_year / 365)
	]

for i in range(longitud):
    X_new = crear_ventanas(forecasting.loc[forecasting.index[i]:forecasting.index[i+ventana], :].copy())
    pred = model.predict(X_new.drop(columns=['fecha', 'afluencia_tren_ligero']))[0]
    forecasting.loc[forecasting.fecha == pd.to_datetime(forecasting.loc[forecasting.index[i+ventana], 'fecha']), 'afluencia_tren_ligero'] = pred

C:\Users\Ian\AppData\Local\Temp\ipykernel_19040\1838942369.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [12]:
# Graficamos
fig = px.line(forecasting[forecasting.fecha <= df_train.fecha.max()][['fecha', 'afluencia_tren_ligero']],
            x='fecha', y='afluencia_tren_ligero',
			title=f'Forecasting {longitud} dias')
fig.data[0].name = 'Pasado'
fig.add_scatter(
    x=forecasting[forecasting.fecha >= df_train.fecha.max()]['fecha'],
    y=forecasting[forecasting.fecha >= df_train.fecha.max()]['afluencia_tren_ligero'],
    mode='lines',
    name='Forecasting'
)
fig.show()